In [1]:
!pip install --force-reinstall --no-cache-dir numpy==1.23.5 scipy==1.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 146.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 105.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires scipy>=1.11.4, but you have scipy 1.10.1 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires sc

In [1]:
!pip install gensim==4.3.1
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
t

In [2]:
import pandas as pd

balanced_wine_reviews = pd.read_csv('balanced_wine_reviews.csv')
balanced_wine_reviews.head()

,variety,description
0,Bordeaux-style Red Blend,This structured wine has 25-year old vines as ...
1,Bordeaux-style Red Blend,This is a great wine. It has all the elements ...
2,Bordeaux-style Red Blend,"This is a fruity, Cabernet Sauvignon-dominated..."
3,Bordeaux-style Red Blend,Merlot makes up three quarters of this wine wi...
4,Bordeaux-style Red Blend,"This wine shows extraction, new wood and firm ..."


## Word2Vec

In [3]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('english'))

def tokenize_text(text):
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

balanced_wine_reviews["tokens"] = balanced_wine_reviews["description"].apply(tokenize_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
from gensim.models import Word2Vec
SEED=42
# Train Word2Vec on your tokenized corpus
w2v_model = Word2Vec(
    sentences=balanced_wine_reviews["tokens"].tolist(),
    vector_size=100,
    window=5,
    min_count=2,
    workers=4,
    seed=SEED
)

In [5]:
import numpy as np

def get_avg_embedding(tokens, model, vector_size=100):
    valid_words = [word for word in tokens if word in model.wv]
    if not valid_words:
        return np.zeros(vector_size)
    return np.mean([model.wv[word] for word in valid_words], axis=0)

# Create embedding matrix
X_vectors = np.vstack([
    get_avg_embedding(tokens, w2v_model, vector_size=100)
    for tokens in balanced_wine_reviews["tokens"]
])

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_vectors,
    balanced_wine_reviews["variety"],
    test_size=0.2,
    random_state=SEED,
    stratify=balanced_wine_reviews["variety"]
)

clf = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=SEED)
clf.fit(X_train, y_train)

# Predict
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

                          precision    recall  f1-score   support

Bordeaux-style Red Blend       0.52      0.58      0.55       200
      Cabernet Sauvignon       0.49      0.36      0.42       200
              Chardonnay       0.52      0.54      0.53       200
                  Merlot       0.40      0.30      0.34       200
              Pinot Noir       0.38      0.34      0.36       200
               Red Blend       0.48      0.50      0.49       200
                Riesling       0.53      0.67      0.59       200
                    Rosé       0.52      0.55      0.53       200
         Sauvignon Blanc       0.48      0.42      0.45       200
                   Syrah       0.39      0.47      0.42       200

                accuracy                           0.47      2000
               macro avg       0.47      0.47      0.47      2000
            weighted avg       0.47      0.47      0.47      2000

